# 📝 Building a Notion Integration with Python 🚀

Hey there! Welcome to this fun guide where we'll learn how to connect your Python applications with Notion! We'll create a powerful integration that lets you programmatically interact with your Notion workspace. 🌟

## 🎯 What We'll Build

We're going to create a Notion integration that can:
1. 📊 Read and write to Notion databases
2. 📑 Create and update pages
3. 🔄 Sync data between Notion and your applications

## ✅ Prerequisites

Before we dive in, make sure you have:
- 📓 A Notion account
- 🎯 A database in Notion that you want to work with

## 🔑 Part 1: Setting Up Your Notion Integration

First, let's get you set up with the necessary credentials to talk to Notion! 

### 1. Create a Notion Integration

1. 🌐 Go to [Notion's Integration page](https://www.notion.so/my-integrations)
2. 👆 Click the "New integration" button
3. 📝 Give your integration a name (like "My Python Integration")
4. 🎨 Choose an icon and color if you want (make it pretty! ✨)
5. 💫 Select the workspace where you want to use the integration
6. 🎁 Click "Submit" to create your integration

You'll receive a secret token that looks something like this:
`secret_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx`

⚠️ Important Security Note! ⚠️
Keep this token safe and never share it publicly! It's like the key to your Notion kingdom! 🏰

### 2. Get Your Database ID

Now we need to find the ID of the database you want to work with:

1. 📊 Open your Notion database in the browser
2. 🔗 Look at the URL, it will look something like:
   `https://www.notion.so/workspace/[database-id]?v=...`
3. 📋 Copy the database ID part (it's a string of characters between the last '/' and '?')

For example, from this URL:
`https://www.notion.so/workspace/7749c8c4a3f34c8c8c8c8c8c8c8c8c8c?v=...`
The database ID would be: `7749c8c4a3f34c8c8c8c8c8c8c8c8c8c`

### 3. Connect Your Database

One last important step!

1. 📱 Go to your database in Notion
2. ⚙️ Click the '...' menu in the top right
3. 👥 Look for "Add connections"
4. 🔍 Find your integration and click to connect
5. ✅ Click "Confirm" to give your integration access

Now your integration has permission to access this database! 🎉

## 🛠️ Part 2: Setting Up Your Python Environment

Let's get your Python environment ready with all the tools we need! We'll use the official Notion SDK for Python to make our lives easier. 🐍

First, let's install the required package:

In [1]:
# https://www.notion.so/19fb6e8bf55080f086fde76aeb1d4cec?v=19fb6e8bf55080d280af000ccc966e56&pvs=4

In [2]:
#!pip install notion-client openai

In [18]:
import os
from openai import OpenAI
from notion_client import Client
from dotenv import load_dotenv
from pprint import pprint # Pretty printing json

load_dotenv()

True

## 🔐 Part 3: Setting Up Environment Variables

Let's keep our secrets safe! We'll use a `.env` file to store our sensitive information. 

Create a file named `.env` in your project directory and add these lines:

In [19]:
NOTION_API_KEY = "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"
TRAVEL_DB_ID = "19fb6e8bf55080f086fde76aeb1d4cec"

## 📊 Part 4: Basic Notion Operations

Let's look at some common operations you can do with your Notion integration! Each operation is like a different superpower for your application! 🦸‍♂️

### Reading from a Database 📖

The Notion API lets you query your database like a pro! You can:
- 🔍 Filter entries based on properties
- 📋 Sort entries in any order
- 📝 Get specific properties only

In [20]:

client = Client(auth=NOTION_API_KEY)

response = client.databases.query(
    database_id=TRAVEL_DB_ID,
)

response

{'object': 'list',
 'results': [{'object': 'page',
   'id': '19fb6e8b-f550-8167-8d42-f7939b265882',
   'created_time': '2025-02-19T16:42:00.000Z',
   'last_edited_time': '2025-02-19T16:42:00.000Z',
   'created_by': {'object': 'user',
    'id': 'bbe31a52-f33e-46f4-b8bc-1f9e968168e1'},
   'last_edited_by': {'object': 'user',
    'id': 'bbe31a52-f33e-46f4-b8bc-1f9e968168e1'},
   'cover': None,
   'icon': None,
   'parent': {'type': 'database_id',
    'database_id': '19fb6e8b-f550-80f0-86fd-e76aeb1d4cec'},
   'archived': False,
   'in_trash': False,
   'properties': {'Landscape Types': {'id': '%3AZDy',
     'type': 'multi_select',
     'multi_select': [{'id': 'pECU', 'name': 'Beach', 'color': 'blue'},
      {'id': '|mCe', 'name': 'Island', 'color': 'blue'},
      {'id': '4d02a680-b7a8-4869-b5db-5e358a23e531',
       'name': 'Forest',
       'color': 'green'}]},
    'Currency': {'id': 'RUM%7C',
     'type': 'rich_text',
     'rich_text': [{'type': 'text',
       'text': {'content': 'Seychel

In [ ]:
# Fetch the database schema
response = client.databases.retrieve(TRAVEL_DB_ID)

# Extract the Budget select options and their IDs
budget_property = response["properties"].get("Budget")
if budget_property and "select" in budget_property:
    budget_options = budget_property["select"]["options"]
    budget_mapping = {option["name"]: {"name": option["name"], "id": option["id"]} for option in budget_options}

    print("Budget Mapping:", budget_mapping)

In [ ]:
pages = []

for page in response["results"]:
    properties = page["properties"]
    page_dict = {
        "name": properties.get("Name", {}).get('title', [{}])[0].get('plain_text', ""),
        "priority": (properties.get("Priority", {}).get('select') or {}).get('name', ""),
        "deadline": (properties.get("Deadline", {}).get('date') or {}).get('start', ""),
        "status": (properties.get("Status", {}).get('status') or {}).get('name', ""),
        "size": (properties.get("Size", {}).get('select') or {}).get('name', ""),
        "area": (properties.get("Area", {}).get('select') or {}).get('name', ""),
    }
    pages.append(page_dict)
pprint(pages)

In [ ]:
async def get_activities(status_filter="not_done"):
    """
    Fetch activities from Notion database with optional status filtering
    
    Parameters:
        status_filter (str): Filter type for status. Options:
            - "not_done": Returns activities not marked as Done (default)
            - "done": Returns only Done activities
            - "all": Returns all activities regardless of status
            
    Returns:
        list: List of dictionaries containing activity details
    """
    try:
        # 🔍 Prepare filter based on status_filter parameter
        filter_params = {}
        if status_filter == "not_done":
            filter_params = {
                "property": "Status",
                "status": {
                    "does_not_equal": "Done"
                }
            }
        elif status_filter == "done":
            filter_params = {
                "property": "Status",
                "status": {
                    "equals": "Done"
                }
            }
        
        # 📤 Query the database
        query_params = {"database_id": TRAVEL_DB_ID}
        if filter_params:
            query_params["filter"] = filter_params
            
        response = client.databases.query(**query_params)
        
        # 📋 Process results
        pages = []
        for page in response["results"]:
            properties = page["properties"]
            page_dict = {
                "name": properties.get("Name", {}).get('title', [{}])[0].get('plain_text', ""),
                "priority": (properties.get("Priority", {}).get('select') or {}).get('name', ""),
                "deadline": (properties.get("Deadline", {}).get('date') or {}).get('start', ""),
                "status": (properties.get("Status", {}).get('status') or {}).get('name', ""),
                "size": (properties.get("Size", {}).get('select') or {}).get('name', ""),
                "area": (properties.get("Area", {}).get('select') or {}).get('name', ""),
            }
            pages.append(page_dict)
            
        return pages
        
    except Exception as e:
        print(f"❌ Error fetching activities from Notion: {str(e)}")
        return []

# 🎯 Example usage:
# Get all non-completed activities
active_activities = await get_activities()

# Get all completed activities
completed_activities = await get_activities(status_filter="done")

# Get all activities
all_activities = await get_activities(status_filter="all")

In [ ]:
all_activities

### Writing to a Database ✍️

You can create new pages in your database with:
- 📝 Text content
- ✅ Checkboxes
- 📅 Dates
- 👥 People mentions
- And many more property types!

In [16]:
async def write_activity(location, language, currency, landscape_types, best_months_to_visit, budget, food, activities):
    """
    Create a new activity in the Notion database
    
    Parameters:
        Country Name (str): Name of the country
        National Language (str): National language of the country
        Currency (str): Currency of the country
        Landscape Types (str): Landscape types of the country
        Best Months to Visit (str): Best months to visit the country
        Budget Range (str): Budget range for the country
        Food (str): Food in the country
        Activities (str): Activities to do in the country
        
    Returns:
        dict: Created page object or None if failed
    """

    properties = {}


    budget_mapping = {'€€€': {'name': '€€€', 'id': 'iBKY'}, '€€': {'name': '€€', 'id': 'uKer'}, '€': {'name': '€', 'id': 'WqyF'}}

    
    try:
        # 📋 Prepare the properties for the new activity
        # properties = {
        #     "Budget": {
        #         "id": "budget_property_id",
        #         "type": "select",
        #         "select": {
        #                 "options": [{"id": "id_for_$","name": "$"},{"id": "id_for_$$","name": "$$"},{"id": "id_for_$$$","name": "$$$"}]
        #         }
        #     }
        #}
        
        # Add optional properties if provided
        if location:
            properties["Location"] = {
                "title": [{"text": {"content": location}}]  # Correct title format
            }
        
        if language:
            existing_languages = ["French", "Korean", "English", "Spanish", "Portuguese", "Italian", "Japanese", "Russian", "Arabic", "Turkish"]  # Predefined languages (can be fetched manually from your Notion database)
            language_list = language.split(",")  # Assume you get a comma-separated list from your web search

            selected_languages = [
                {"name": lang.strip()} if lang.strip() in existing_languages else {"name": "Other"}
                for lang in language_list
            ]
    
            properties["Language"] = {
                "multi_select": selected_languages
            }
        
        if currency:
            properties["Currency"] = {
                "rich_text": [{"text": {"content": currency}}]  # Currency as text field
            }
        
        if landscape_types:
            properties["Landscape Types"] = {
                "multi_select": [{"name": landscape_type} for landscape_type in landscape_types]  # Correct multi_select format
            }
        
        if best_months_to_visit:
            properties["Best Months to Visit"] = {
                "multi_select": [{"name": month} for month in best_months_to_visit]  # Correct multi_select format
            }
        
        if budget:
            if isinstance(budget, list):  # Handle case where budget is a list
                budget = budget[0]

            properties["Budget"] = {"select": budget_mapping.get(budget, {"name": budget})}

        
        if food:
            properties["Food"] = {
                "rich_text": [{"text": {"content": food}}]  # Correct rich_text format
            }
        
        if activities:
            properties["Activities"] = {
                "rich_text": [{"text": {"content": activities}}]  # Correct rich_text format
            }

        children = [
            {
                "object": "block",
                "type": "heading_3",  # Ensure you are using the correct block type
                "heading_3": {
                    "rich_text": [
                        {
                            "type": "text",
                            "text": {
                                "content": "Activities"
                            }
                        }
                    ]
                }
            },
            {
                "object": "block",
                "type": "paragraph",  # Make sure it's a paragraph block
                "paragraph": {
                    "rich_text": [
                        {
                            "type": "text",
                            "text": {
                                "content": activities
                            }
                        }
                    ]
                }
            },
            {
                "object": "block",
                "type": "heading_3",  # Same for this block
                "heading_3": {
                    "rich_text": [
                        {
                            "type": "text",
                            "text": {
                                "content": "Food"
                            }
                        }
                    ]
                }
            },
            {
                "object": "block",
                "type": "paragraph",  # And here as well
                "paragraph": {
                    "rich_text": [
                        {
                            "type": "text",
                            "text": {
                                "content": food
                            }
                        }
                    ]
                }
            }
        ]    
        
        # 📤 Create the activity in Notion
        new_page = client.pages.create(
            parent={"database_id": TRAVEL_DB_ID},
            properties=properties,
            children=children
        )
        
        print(f"✅ Successfully created activity: {location}")
        return new_page
        
    except Exception as e:
        print(f"❌ Error creating activity in Notion: {str(e)}")
        return None

In [17]:
# 🎯 Example usage:
new_activity = await write_activity(
    location="United States",
    language="English",
    currency="USD",
    landscape_types=["Mountains", "Beaches", "Cities"],
    best_months_to_visit=["June", "July", "August"],
    budget=["€€€"],
    food="""- Matata (a dish often containing palm oil, fish, and vegetables)
    - Calulu (a fish or meat stew)
    - Feijoada (a hearty beans and meat stew)
    - Grilled fish served with rice and vegetables
    - Sonhos de banana (sweet banana fritters)
    """,
    activities="""- Explore Obô National Park for hiking and biodiversity
    - Visit the Claudio Corallo Chocolate Factory for chocolate tasting
    - Relax at Praia Banana and enjoy the beautiful beaches
    - Discover the history at São Sebastiao Museum and the colonial architecture in São Tomé city
    - Go whale watching during the rainy season or turtle watching during nesting seasons
    """
)

❌ Error creating activity in Notion: name 'client' is not defined


In [37]:
import sys
import os


# Define system path as root
sys.path.append("../agent")


import json
from dotenv import load_dotenv
from openai import OpenAI
from tools import TOOLS, search_web
from prompts import SYSTEM_PROMPT
from pydantic import BaseModel, Field


load_dotenv()

class TripSummary(BaseModel):
    summary: str = Field(description="The summary of the trip, including the information in all other keys.")
    location: str = Field(description="The name of the location.")
    language: str = Field(description="The main language spoken in the area.")
    currency: str = Field(description="The local currency used.")
    landscape_types: list[str] = Field(description="A list of landscape types (e.g., desert, mountain, forest).")
    best_months_to_visit: list[str] = Field(description="A list of months that are best for visiting.")
    budget: str = Field(description="The typical cost level for a visit (e.g., €€€, €€).")
    food: str = Field(description="Common or local food in the area. Extend")
    activities: str = Field(description="Notable activities to do in the area.")


def trip_creator(messages):
    client = OpenAI()
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=messages,
        response_format=TripSummary,
    )
    return completion.choices[0].message.parsed

def agent(messages):

    # Initialize the OpenAI client
    client = OpenAI()

    # Make the OpenAI API call to extract the events
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOLS
    )

    # Parse the response
    response = completion.choices[0].message

    agent_response = {
        "content": "",
        "trip_summary": {}
    }

    # Parse the response to get the tool call arguments
    if response.tool_calls:
        # Process each tool call
        for tool_call in response.tool_calls:
            # Get the tool call arguments
            tool_call_arguments = json.loads(tool_call.function.arguments)
            if tool_call.function.name == "search_web":
                print(f"Searching the web: {tool_call_arguments['query']}")
                search_results = search_web(tool_call_arguments["query"])
                messages.append({"role": "assistant", "content": f"{tool_call_arguments["query"]}: {search_results}"})
        print(f"Creating trip summary: {tool_call_arguments}")
        trip_summary = trip_creator(messages)
        agent_response["trip_summary"] = json.loads(trip_summary.model_dump_json())
    else:
        # If there are no tool calls, return the response content
        agent_response["content"] = response.content
    return agent_response
   # return pd.DataFrame([json.loads(event.model_dump_json()) for event in response.events])

In [38]:
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "I want to go to Rome"}
]

In [39]:
response = agent(messages)
if response["content"] != "":
    messages.append({"role": "assistant", "content": response["content"]})
else:
    messages.append({"role": "assistant", "content": response["trip_summary"]})
messages

[{'role': 'system',
  'content': "\n- You are a helpful assistant that can search the web to educate the user about a location for a travel wishlist.\n- Use the search_web function to search the web for information on the trips.\n- You should always ask 2 clarification questions to the user to understand the user's needs better BEFORE makeing any tools calls.\n- The questions should collect information about the location, the user's preferences, and the user's budget. Find an example below.\n- Once you have all the information from the user, make search_web tool calls to get the trip summary and make a final tool call to the create_trip_summary tool to create the trip summary.\n\nExample of tool_calls list: [\n- Searching the web: Rome travel information summer museums restaurants budget  \n- Searching the web: Rome best months to visit museums restaurants\n- Searching the web: Rome local food types and budget dining options\n- Create trip summary: Rome, summer, museums, restaurants, b

In [40]:
messages.append({"role": "user", "content": "I want to go during the summer and I'm traveling alone"})

In [41]:
response = agent(messages)
if response["content"] != "":
    messages.append({"role": "assistant", "content": response["content"]})
else:
    messages.append({"role": "assistant", "content": response["trip_summary"]})
messages

[{'role': 'system',
  'content': "\n- You are a helpful assistant that can search the web to educate the user about a location for a travel wishlist.\n- Use the search_web function to search the web for information on the trips.\n- You should always ask 2 clarification questions to the user to understand the user's needs better BEFORE makeing any tools calls.\n- The questions should collect information about the location, the user's preferences, and the user's budget. Find an example below.\n- Once you have all the information from the user, make search_web tool calls to get the trip summary and make a final tool call to the create_trip_summary tool to create the trip summary.\n\nExample of tool_calls list: [\n- Searching the web: Rome travel information summer museums restaurants budget  \n- Searching the web: Rome best months to visit museums restaurants\n- Searching the web: Rome local food types and budget dining options\n- Create trip summary: Rome, summer, museums, restaurants, b

In [42]:
messages.append({"role": "user", "content": "Museums and restaurants, low budget"})

In [43]:
response = agent(messages)
if response["content"] != "":
    messages.append({"role": "assistant", "content": response["content"]})
else:

    messages.append({"role": "assistant", "content": response["trip_summary"]["summary"]})
    print(response["trip_summary"]["summary"])
    new_activity = await write_activity(
        location=response["trip_summary"]["location"],
        language=response["trip_summary"]["language"],
        currency=response["trip_summary"]["currency"],
        landscape_types=response["trip_summary"]["landscape_types"],
        best_months_to_visit=response["trip_summary"]["best_months_to_visit"],
        budget=response["trip_summary"]["budget"],
        food=response["trip_summary"]["food"],
        activities=response["trip_summary"]["activities"]
    )
    
messages

Searching the web: Rome travel information summer museums restaurants budget
Searching the web: Rome best months to visit museums restaurants
Searching the web: Rome local food types and budget dining options
Creating trip summary: {'query': 'Rome local food types and budget dining options'}
Explore Rome in the summer with a focus on museums and local restaurants while keeping to a budget. Experience the rich culture through its numerous museums and savor authentic Roman cuisine at budget-friendly eateries. With average daily costs around €88, you can enjoy the city without overspending.
✅ Successfully created activity: Rome


[{'role': 'system',
  'content': "\n- You are a helpful assistant that can search the web to educate the user about a location for a travel wishlist.\n- Use the search_web function to search the web for information on the trips.\n- You should always ask 2 clarification questions to the user to understand the user's needs better BEFORE makeing any tools calls.\n- The questions should collect information about the location, the user's preferences, and the user's budget. Find an example below.\n- Once you have all the information from the user, make search_web tool calls to get the trip summary and make a final tool call to the create_trip_summary tool to create the trip summary.\n\nExample of tool_calls list: [\n- Searching the web: Rome travel information summer museums restaurants budget  \n- Searching the web: Rome best months to visit museums restaurants\n- Searching the web: Rome local food types and budget dining options\n- Create trip summary: Rome, summer, museums, restaurants, b

In [21]:
new_activity = await write_activity(
    location=response["trip_summary"]["location"],
    language=response["trip_summary"]["language"],
    currency=response["trip_summary"]["currency"],
    landscape_types=response["trip_summary"]["landscape_types"],
    best_months_to_visit=response["trip_summary"]["best_months_to_visit"],
    budget=response["trip_summary"]["budget"],
    food=response["trip_summary"]["food"],
    activities=response["trip_summary"]["activities"]
)
new_activity

KeyError: 'trip_summary'

In [46]:
def generate_cover_image(prompt):
    client = OpenAI()
    response = client.images.generate(
        prompt=prompt,
        n=1,
        size="1024x1024"
    )
    return response.data[0].url

In [47]:
generate_cover_image("A cover image ok Tokyo")

'https://oaidalleapiprodscus.blob.core.windows.net/private/org-JtS2wjPGbVfl4fReZyyF0CkX/user-tIUeaJQZJ8gKXh3Wwk6PzNkV/img-oEZMCe8jBbssccY2PdlK1mSh.png?st=2025-02-20T14%3A51%3A01Z&se=2025-02-20T16%3A51%3A01Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=d505667d-d6c1-4a0a-bac7-5c84a87759f8&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2025-02-20T00%3A40%3A16Z&ske=2025-02-21T00%3A40%3A16Z&sks=b&skv=2024-08-04&sig=resyxE7c0/SrLlRziXUFxco1UY9dPDLRJYnfzZN%2B3bw%3D'

## 🎨 Part 5: Understanding Notion Properties

Notion uses different property types for different kinds of data. Here are the main ones you'll work with:

### Common Property Types 📋

1. **Text** 📝
   - Title
   - Rich Text
   - URL

2. **Numbers & Dates** 🔢
   - Number
   - Date
   - Created time
   - Last edited time

3. **Organization** 🗂️
   - Select
   - Multi-select
   - Status
   - Files & Media

4. **People** 👥
   - Person
   - Created by
   - Last edited by

### Property Format Tips 💡

- 📅 Dates should be in ISO format: `2024-03-21`
- ✨ Select options must exist in the database
- 👤 People are referenced by their Notion user IDs
- 🔗 URLs must include `https://` or `http://`

## 🌟 Best Practices & Tips

Here are some pro tips to make your Notion integration awesome:

### Performance Tips 🚀

1. **Batch Operations** 📦
   - Group multiple updates together
   - Use bulk operations when possible
   - Cache results when appropriate

2. **Rate Limits** ⏱️
   - Notion has rate limits
   - Add delay between requests
   - Handle rate limit errors gracefully

### Error Handling 🛡️

Always prepare for these common scenarios:
- 🔒 Authentication errors
- 📡 Network issues
- 🚫 Permission problems
- ⏳ Rate limiting

### Security Best Practices 🔐

1. **Keep Secrets Safe** 
   - Use environment variables
   - Never expose your integration token
   - Regularly rotate tokens

2. **Access Control**
   - Only request necessary permissions
   - Regularly audit database access
   - Remove unused integrations

Remember: A well-structured integration is a happy integration! 🎉